In [ ]:
# !pip install transformers torch

In [ ]:
import pandas as pd

from google.colab import drive
import os
import shutil
import zipfile

drive.mount("/content/gdrive/") # 지정 위치와 구글 드라이브 연결

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline
import torch
from tqdm.auto import tqdm

In [ ]:
file_path = '/content/gdrive/MyDrive/GMSW/project/result/'
df = pd.read_csv(file_path+'news_topic_df_ssk.csv')

## 감성분석

### 감성 분류

In [ ]:
# GPU 설정 (Colab 사용 시 필수! 속도 10배 차이)
device = 0 if torch.cuda.is_available() else -1
print(f"사용 장치: {'GPU (빠름)' if device == 0 else 'CPU (느림, 인내심 필요)'}")

In [ ]:
# 금융 특화 모델 로드 (snunlp/KR-FinBert-SC): 서울대 NLP 랩에서 만든 한국어 금융 감성분석 모델
model_name = "snunlp/KR-FinBert-SC"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# 고속 파이프라인 생성 (batch_size=32 추천)
sentiment_analyzer = pipeline(
    "sentiment-analysis",
    model=model,
    tokenizer=tokenizer,
    device=device, # GPU 사용
    batch_size=32, # 한 번에 32개씩 처리 (속도 핵심!)
    truncation=True,
    max_length=128 # 제목이니까 128 토큰이면 충분
)

# 분석 실행
titles = df['title'].tolist()
results = []

for output in tqdm(sentiment_analyzer(titles), total=len(titles)):
    results.append(output)

# 결과 저장
# label(positive/negative/neutral)과 score(확신도) 분리
df['Sentiment_Label'] = [res['label'] for res in results]
df['Sentiment_Score'] = [res['score'] for res in results]

# Label을 숫자로 변환: positive -> 1, neutral -> 0, negative -> -1
label_map = {'positive': 1, 'neutral': 0, 'negative': -1}
df['Sentiment_Num'] = df['Sentiment_Label'].map(label_map)

In [ ]:
# # 파일 저장
# df.to_csv("/content/gdrive/MyDrive/GMSW/project/result/news_topic_sentiment_result_ssk.csv", index=False)

### 분석

In [ ]:
file_path = '/content/gdrive/MyDrive/GMSW/project/result/'
df = pd.read_csv(file_path+'news_topic_sentiment_result_ssk.csv')

In [ ]:
df.head()

In [ ]:
topics = [0, 1, 2, 3, 4, 5]

df_clean = df[df['topic'].isin(topics)].copy()
df_clean = df_clean[df_clean['Sentiment_Score']>=0.5].copy()

print(f"원본 데이터: {len(df)}개 -> 정제된 데이터: {len(df_clean)}개")

In [ ]:
import plotly.express as px

# 1. 토픽별 평균 감성 점수 계산
# (Sentiment_Num은 긍정=1, 중립=0, 부정=-1로 변환된 수치라고 가정)
# 만약 KR-FinBERT Score(0~1)라면 Sentiment_Label을 참고하여 -1~1로 스케일링 필요
# Sentiment_Score(확신도)가 아니라 긍/부정 라벨을 수치화한 것을 쓴다고 가정

# Sentiment_Num: 긍정(1), 중립(0), 부정(-1)
topic_sentiment = df_clean.groupby('Topic_Name')['Sentiment_Num'].mean().reset_index()
topic_sentiment = topic_sentiment.sort_values('Sentiment_Num', ascending=True)

# 2. 시각화 (Bar Chart)
fig = px.bar(topic_sentiment,
             x='Sentiment_Num',
             y='Topic_Name',
             orientation='h',
             title='<b>토픽별 평균 감성 점수 (Market Sentiment by Topic)</b>',
             text_auto='.3f',
             color='Sentiment_Num',
             color_continuous_scale='RdBu') # 빨강(부정) ~ 파랑(긍정)

fig.update_layout(xaxis_title="평균 감성 점수 (-1: 부정, +1: 긍정)", yaxis_title="")
fig.show()

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

stock = pd.read_csv('/content/gdrive/MyDrive/GMSW/project/data/stock_price.csv')
# stock.head()

# Min-Max 정규화
def min_max_scale(series):
    return (series - series.min()) / (series.max() - series.min())

# 정규화된 컬럼 생성
stock['Scaled_samsung_price'] = min_max_scale(stock['samsung_price'])
stock['Scaled_skhynix_price'] = min_max_scale(stock['skhynix_price'])

# 1. 일별 평균 감성 점수 계산
daily_sentiment = df_clean.groupby('date')['Sentiment_Num'].mean()
# 이동평균(MA)을 적용하면 노이즈를 줄이고 추세를 보기 좋습니다 (3일 or 5일)
daily_sentiment_ma = daily_sentiment.rolling(window=5).mean()

# 2. 기존 이중축 코드 활용
fig = make_subplots(specs=[[{"secondary_y": True}]])

# [왼쪽] 뉴스 감성 점수 (이동평균)
fig.add_trace(
    go.Scatter(x=daily_sentiment_ma.index, y=daily_sentiment_ma.values,
               name="뉴스 감성 지수 (5일 이평)",
               line=dict(color='purple', width=1)
              ),
    secondary_y=False
)

# [오른쪽] SK하이닉스 주가 (Scaled)
fig.add_trace(
    go.Scatter(x=stock['date'], y=stock['Scaled_skhynix_price'],
               name="SK하이닉스 (Scaled)", line=dict(color='royalblue', dash='dot')),
    secondary_y=True
)

fig.add_trace(
    go.Scatter(x=stock['date'], y=stock['Scaled_samsung_price'],
               name="삼성 (Scaled)", line=dict(color='red', dash='dot')),
    secondary_y=True
)

fig.update_layout(title="<b>뉴스 감성 변화와 주가 추세</b>",
                  legend=dict(orientation="h", y=1.1, x=0, xanchor='left'), # 범례 위로 올리기
                  )
fig.show()

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# ---------------------------------------------------------
# [사전 설정] 토픽 분류 (사용자 데이터의 토픽 이름과 일치해야 합니다)
# ---------------------------------------------------------
# df_clean의 토픽 이름이 들어있는 컬럼명 (예: 'Name', 'Topic_Name', 'Custom_Name' 등 확인 필요)
topic_col = 'Topic_Name'  # <-- 본인 데이터 컬럼명으로 수정하세요!

# 1. 실적/펀더멘털 관련 토픽 리스트
earnings_topics = ['산업트렌드/실적/펀더멘털']
# (시황/랠리도 상승 재료로 본다면 포함, 순수 실적만 보려면 '시황/랠리' 제외)

# 2. 정치/거시/정책 관련 토픽 리스트
# politics_topics = ['글로벌 거시 정치', '정책']
politics_topics = ['협력 기대감']

# ---------------------------------------------------------
# [데이터 전처리]
# ---------------------------------------------------------
# 주가 데이터 로드 및 스케일링 (기존 코드 동일)
stock = pd.read_csv('/content/gdrive/MyDrive/GMSW/project/data/stock_price.csv')

def min_max_scale(series):
    return (series - series.min()) / (series.max() - series.min())

stock['Scaled_samsung_price'] = min_max_scale(stock['samsung_price'])
stock['Scaled_skhynix_price'] = min_max_scale(stock['skhynix_price'])

# ---------------------------------------------------------
# [감성 점수 분리 계산]
# ---------------------------------------------------------

# 1. 실적(Earnings) 관련 뉴스 필터링 및 5일 이평 계산
mask_earnings = df_clean[topic_col].isin(earnings_topics)
daily_earnings = df_clean[mask_earnings].groupby('date')['Sentiment_Num'].mean()
daily_earnings_ma = daily_earnings.rolling(window=5).mean()

# 2. 정치(Politics/Macro) 관련 뉴스 필터링 및 5일 이평 계산
mask_politics = df_clean[topic_col].isin(politics_topics)
daily_politics = df_clean[mask_politics].groupby('date')['Sentiment_Num'].mean()
daily_politics_ma = daily_politics.rolling(window=5).mean()

# ---------------------------------------------------------
# [시각화]
# ---------------------------------------------------------
fig = make_subplots(specs=[[{"secondary_y": True}]])
colors = ['#FFA07A', '#20B2AA', '#87CEFA', '#fbe383']

# [왼쪽 축 1] 실적/펀더멘털 감성 (진한 초록색 - 긍정적 이미지)
fig.add_trace(
    go.Scatter(x=daily_earnings_ma.index, y=daily_earnings_ma.values,
               name="실적/펀더멘털 감성 (5일 이평)",
               line=dict(color='#87CEFA', width=1),
               fill='tozeroy', opacity=0.3
               ),
    secondary_y=False
)

# [왼쪽 축 2] 정치/거시 감성 (진한 빨간색 - 경고/불확실성 이미지)
fig.add_trace(
    go.Scatter(x=daily_politics_ma.index, y=daily_politics_ma.values,
               name="협력 기대감 (5일 이평)",
               line=dict(color='#FFA07A', width=1),
               fill='tozeroy', opacity=0.3
               ),
    secondary_y=False
)

# [오른쪽 축] 주가 데이터 (점선으로 처리하여 감성 지수와 구분)
fig.add_trace(
    go.Scatter(x=stock['date'], y=stock['Scaled_skhynix_price'],
               name="SK하이닉스 (Scaled)",
               line=dict(color='royalblue', dash='dot', width=1.5)),
    secondary_y=True
)

fig.add_trace(
    go.Scatter(x=stock['date'], y=stock['Scaled_samsung_price'],
               name="삼성전자 (Scaled)",
               line=dict(color='red', dash='dot', width=1.5)),
    secondary_y=True
)

# 레이아웃 설정
fig.update_layout(
    title="<b>뉴스 토픽별(실적 vs 협력 기대감) 감성 변화와 주가 추세 비교</b>",
    template='plotly_white',
    legend=dict(orientation="h", y=1.1, x=0.5, xanchor='center'),
    xaxis_title="날짜",
    yaxis_title="감성 점수 (실선)",
    yaxis2_title="주가 (점선, Scaled)",
    hovermode="x unified" # 마우스 오버 시 모든 데이터 함께 보기
)

# y축 범위 고정 (감성 점수 비교 용이하게)
fig.update_yaxes(range=[-1, 1], secondary_y=False) # 감성 점수는 -1 ~ 1 사이

fig.show()

### 검증

In [ ]:
import pandas as pd
from scipy import stats
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
# 이벤트 시작일 지정
event_start_date = "2025-10-16"

# 날짜 형식 변환
df_clean['date'] = pd.to_datetime(df_clean['date'])
threshold_date = pd.to_datetime(event_start_date)

# Pre: 이벤트 이전 / Post: 이벤트 이후 (급등 구간)
group_pre = df_clean[df_clean['date'] < threshold_date]
group_post = df_clean[df_clean['date'] >= threshold_date]

print(f"분석 기간 설정: {event_start_date} 기준")
print(f"Pre-Event 표본 수: {len(group_pre)}개")
print(f"Post-Event 표본 수: {len(group_post)}개")

In [ ]:
# T-test 1: 감성 점수 (Sentiment)
# "급등 후에 뉴스가 더 긍정적으로 변했는가?"
t_stat_sent, p_val_sent = stats.ttest_ind(
    group_post['Sentiment_Num'],
    group_pre['Sentiment_Num'],
    equal_var=False # 등분산 가정 X (Welch's t-test) -> 더 안전한 방법
)

# T-test 수행 2: 관심도/보도량 (Volume)
# "급등 후에 하루에 쏟아지는 기사량이 늘었는가?"
# 일별 기사 수로 집계해야 함
vol_pre = group_pre.groupby('date').size()
vol_post = group_post.groupby('date').size()

t_stat_vol, p_val_vol = stats.ttest_ind(
    vol_post,
    vol_pre,
    equal_var=False
)

In [ ]:
# 결과 리포트 출력
print("\n" + "="*50)
print("📢 [최종 가설 검증 결과 리포트]")
print("="*50)

print(f"\n1. 감성 점수 차이 (H1 검증)")
print(f"   - Pre 평균: {group_pre['Sentiment_Num'].mean():.4f}")
print(f"   - Post 평균: {group_post['Sentiment_Num'].mean():.4f}")
print(f"   - T-statistic: {t_stat_sent:.4f}")
print(f"   - P-value: {p_val_sent:.4e} ({'⭐⭐ 유의함' if p_val_sent < 0.05 else '유의하지 않음'})")

print(f"\n2. 관심도(보도량) 차이 (H2 기초)")
print(f"   - Pre 일평균 보도량: {vol_pre.mean():.1f}건")
print(f"   - Post 일평균 보도량: {vol_post.mean():.1f}건")
print(f"   - T-statistic: {t_stat_vol:.4f}")
print(f"   - P-value: {p_val_vol:.4e} ({'⭐⭐ 유의함' if p_val_vol < 0.05 else '유의하지 않음'})")
print("="*50)

In [ ]:
# 시각화를 위한 히스토그램 (Histogram)
# barmode='overlay': 두 그래프를 겹쳐서 비교
# histnorm='probability density': 데이터 개수가 달라도 비율로 비교 가능하게 함

df_clean['Period'] = df_clean['date'].apply(
    lambda x: 'Post-Event' if x >= threshold_date else 'Pre-Event'
)

fig = px.histogram(df_clean,
                   x="Sentiment_Num",
                   color="Period",
                   barmode="overlay",
                   nbins=20, # 막대 개수 조절
                   histnorm='probability density', # 핵심: 개수가 아닌 '밀도'로 비교
                   title="<b>이벤트 전후 감성 점수 밀도 비교 (Histogram)</b>",
                   color_discrete_map={'Pre-Event': 'gray', 'Post-Event': 'red'},
                   opacity=0.6, # 투명도 조절
                  )

fig.update_layout(
    xaxis_title="감성 점수 (-1: 부정, 0: 중립, +1: 긍정)",
    yaxis_title="비중 (Density)",
    template="plotly_white",
    legend=dict(orientation="h", y=1.1, x=0, xanchor='left'), # 범례 위로 올리기
)

# 평균선 추가 (이게 핵심!)
# Pre 평균선 (회색 점선)
fig.add_vline(x=0.2697, line_width=3, line_dash="dash", line_color="gray", annotation_text="Pre")
# Post 평균선 (빨강 실선)
fig.add_vline(x=0.2349, line_width=3, line_dash="solid", line_color="red", annotation_text="Post")

fig.show()

In [ ]:
# 1. 기간(Pre/Post)별 토픽 비중 계산
topic_dist = df_clean.groupby(['Period', 'Topic_Name']).size().reset_index(name='Count')

# 2. 비율(%)로 변환
topic_dist['Percentage'] = topic_dist.groupby('Period')['Count'].transform(lambda x: x / x.sum())

# 3. 시각화 (100% Stacked Bar Chart)
fig = px.bar(topic_dist,
             x='Period',
             y='Percentage',
             color='Topic_Name',
             title="<b>이벤트 전후 뉴스 토픽 구성 변화 (Composition Shift)</b>",
             text_auto='.1%',
             category_orders={"Period": ["Pre-Event", "Post-Event"]} # 순서 고정
            )

fig.update_layout(
    yaxis_title="비중 (%)",
    template="plotly_white",
    legend=dict(orientation="h", y=1.1, x=0, xanchor='left'), # 범례 위로 올리기
)

fig.show()

## 인과분석

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import grangercausalitytests, adfuller

In [ ]:
file_path = '/content/gdrive/MyDrive/GMSW/project/result/'
df = pd.read_csv(file_path+'news_topic_sentiment_result_ssk.csv')
stock = pd.read_csv('/content/gdrive/MyDrive/GMSW/project/data/stock_price.csv')

In [ ]:
# ---------------------------------------------------------
# 2. 데이터 전처리
# ---------------------------------------------------------

# [News] 날짜 형식 변환 및 시간 제거 (YYYY-MM-DD)
df['date'] = pd.to_datetime(df['date']).dt.normalize()

# [News] 토픽별 일일 빈도수 집계 (Pivot Table)
# 주의: PPT에 나온 정확한 '토픽 이름'으로 필터링하거나 이름을 맞춰주세요.
daily_topics = df.pivot_table(index='date', columns='Topic_Name', aggfunc='size', fill_value=0)

In [ ]:
# [Price] 날짜 형식 변환
stock['date'] = pd.to_datetime(stock['date'])
stock.set_index('date', inplace=True)

# [Price] 수익률(Return) 계산 (로그 수익률 추천 - 정상성 확보 유리)
# 두 종목의 평균 수익률을 '반도체 섹터 수익률'로 정의
stock['samsung_ret'] = np.log(stock['samsung_price'] / stock['samsung_price'].shift(1))
stock['skhynix_ret'] = np.log(stock['skhynix_price'] / stock['skhynix_price'].shift(1))
stock['sector_return'] = (stock['samsung_ret'] + stock['skhynix_ret']) / 2

# [Merge] 데이터 병합 (Inner Join으로 교집합 날짜만 분석)
merged_df = pd.merge(daily_topics, stock[['sector_return']], left_index=True, right_index=True, how='inner')

# 결측치 제거 (수익률 계산 첫 날은 NaN 발생하므로 제거)
merged_df.dropna(inplace=True)

print(f"분석 대상 데이터 기간: {merged_df.index.min()} ~ {merged_df.index.max()}")
print(f"총 분석 일수: {len(merged_df)}일")

In [ ]:
merged_df.head()

In [ ]:
# ---------------------------------------------------------
# 3. 그레인저 인과검정 (Granger Causality Test)
# ---------------------------------------------------------

def run_granger_test(data, x_col, y_col, max_lag=3):
    """
    x_col: 원인 변수 (예: 뉴스 빈도)
    y_col: 결과 변수 (예: 주가 수익률)
    """
    print(f"\n[Test] '{x_col}'가 '{y_col}'의 원인인가?")

    # 데이터셋 구성 [종속변수(Y), 독립변수(X)] 순서 주의!
    test_data = data[[y_col, x_col]]

    # 테스트 실행
    # verbose=True로 하면 상세 결과 출력됨
    gc_res = grangercausalitytests(test_data, maxlag=max_lag, verbose=False)

    results = []
    for lag in range(1, max_lag + 1):
        # F-test의 p-value 추출 (ssr_ftest 사용)
        p_value = gc_res[lag][0]['ssr_ftest'][1]
        f_score = gc_res[lag][0]['ssr_ftest'][0]
        results.append({'Lag': lag, 'F-Score': f_score, 'P-Value': p_value})

        significance = "***" if p_value < 0.01 else "**" if p_value < 0.05 else "*" if p_value < 0.1 else ""
        print(f"  Lag {lag}: P-value = {p_value:.4f} {significance}")

    return pd.DataFrame(results)

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

# ---------------------------------------------------------
# 1. 데이터 수집: 모든 토픽의 결과를 하나의 DF로 통합
# ---------------------------------------------------------
results_list = []

# PPT의 핵심 비교군 (컬러 매칭을 위해 순서 중요)
topics_to_test = ['협력 기대감', '산업트렌드/실적/펀더멘털']
existing_topics = [t for t in topics_to_test if t in merged_df.columns]

if not existing_topics:
    print("[!] 데이터에 해당 토픽 컬럼이 없습니다.")
else:
    for topic in existing_topics:
        # 그레인저 인과검정 실행 (이전 단계에서 정의한 함수 사용)
        res_df = run_granger_test(merged_df, x_col=topic, y_col='sector_return', max_lag=5)

        # 'Topic' 컬럼 추가 (어떤 주제의 결과인지 구분)
        res_df['Topic'] = topic
        results_list.append(res_df)

    # 위아래로 데이터 합치기
    final_result_df = pd.concat(results_list, ignore_index=True)

    # ---------------------------------------------------------
    # 2. Plotly Express로 시각화
    # ---------------------------------------------------------

    # 색상 커스텀: 기대감(임팩트) vs 펀더멘털(이성/침착)
    custom_colors = {
        '협력 기대감': '#FF5733',          # 붉은 계열 (강조)
        '산업트렌드/실적/펀더멘털': '#3375FF' # 파란 계열 (대조)
    }

    fig = px.line(
        final_result_df,
        x='Lag',
        y='P-Value',
        color='Topic',
        markers=True,                 # 데이터 포인트 표시
        title='<b>Granger Causality Test</b>: News Topic → Stock Return',
        color_discrete_map=custom_colors, # 지정한 색상 적용
        template='plotly_white'       # 깔끔한 흰색 배경 테마
    )

    # ---------------------------------------------------------
    # 3. 디테일 꾸미기 (PPT용 스타일링)
    # ---------------------------------------------------------

    # (1) 유의수준 0.05 빨간 점선 추가
    fig.add_hline(
        y=0.05,
        line_dash="dash",
        line_color="black",
        annotation_text="Significance Level (0.05)",
        annotation_position="bottom right"
    )

    # (2) 레이아웃 다듬기
    fig.update_layout(
        font=dict(family="Malgun Gothic", size=14), # 폰트 설정
        xaxis=dict(
            title="Time Lag (Days)",
            tickmode='linear', # 1, 2, 3, 4, 5 정수만 표시
            dtick=1
        ),
        yaxis=dict(
            title="P-Value (Lower is More Causal)",
            range=[0, 0.5] # P-value가 너무 튀면 그래프가 납작해지므로 y축 범위 고정 (필요시 조절)
        ),
        legend=dict(
            title=None,          # 범례 제목 제거
            orientation="h",     # 범례 가로로 배치 (그래프 하단)
            yanchor="bottom",
            y=1.02,              # 그래프 위쪽에 배치
            xanchor="right",
            x=1
        ),
        hovermode="x unified"    # 마우스 올렸을 때 비교하기 편하게
    )

    # 그래프 출력
    fig.show()